In [31]:
from jflegDataset import JflegDataset
import pandas as pd
from torch.utils.data import DataLoader
from transformers import BertTokenizer

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
TRAIN_PATH = "src/dataset/train.csv"
VAL_PATH = "src/dataset/eval.csv"
ds = JflegDataset(TRAIN_PATH, tokenizer)
dl = DataLoader(ds)


In [33]:
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
csv = pd.read_csv(TRAIN_PATH)
csv = csv.groupby('input')['target'].agg(np.array).reset_index()
encoded = tokenizer(csv.iloc[4]["input"], return_tensors="pt")
#tokenizer.decode(encoded["input_ids"])
encoded

{'input_ids': tensor([[  101, 12616,   131,   138,  9131,  1107,  1251,  1768,  1209,  1138,
          1618,  2261,  6305,   117,  1104, 16461,  1119,  1209,  1136,  1138,
          1112,  1242,  6305,  1133,  3451,  1122,  1129,   117,  1209,  1129,
          1677,  1618,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [34]:
i = iter(dl)
next(i)
input, targ = next(i)
#ds.decode(input["input_ids"][0])
targ

[{'input_ids': tensor([[  101,   122,   128,   118,   126,   121,  1234,  1127, 13186,  1164,
           1217,  2320,  1106,  1321,  1155,  3553,  1105,  1515, 14930,  3553,
            119,   102]]), 'token_type_ids': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]])},
 {'input_ids': tensor([[  101,   122,   128,   118,   126,   121,  1234,  5980,  1106,  4752,
           1155,  1705,  1105, 14930,  1705,   119,   102]]), 'token_type_ids': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]])},
 {'input_ids': tensor([[  101,   122,   128,   118,   126,   121,  1234,  5980,  1106,  1231,
           4426,  3313,  1155,  1705,  1105, 14930,  1705,   119,   102]]), 'token_type_ids': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]), 'attention_mask': te

In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

ds_train = JflegDataset(TRAIN_PATH, tokenizer)
ds_eval = JflegDataset(VAL_PATH, tokenizer)

dl_train = DataLoader(ds_train)
dl_eval = DataLoader(ds_eval)

tokenizer

BertTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [36]:
from torch import nn, Tensor
import math

# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TokenEmbedding(nn.Module):
    #  https://pytorch.org/tutorials/beginner/translation_transformer.html
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [37]:
import torch
from torch import nn, Tensor

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, 0.1)
        self.embedding = TokenEmbedding(ntoken, d_model)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=4,
            num_encoder_layers=6,
            num_decoder_layers=6,
            dim_feedforward=1024,
            dropout=0.1,
        )

        self.head = nn.Linear(d_model, ntoken)
        self.sm = nn.Softmax(dim=0)

    def forward(self, input: Tensor, target: Tensor, src_mask: Tensor = None) -> Tensor:

        src = self.embedding(input) 
        src = self.pos_encoder(src)

        target = self.embedding(target)
        target = self.pos_encoder(target)

        encoded = self.transformer.encoder(src)

        decoded = self.transformer.decoder(target, memory=encoded)

        out = self.head(decoded)

        return self.sm(out)[:,-1,:] # Consider only the last prediction

In [38]:
model = TransformerModel(tokenizer.vocab_size, 768)

for x,y in dl_train:
    src = x["input_ids"]
    target = y[0]["input_ids"]
    tmp = torch.tensor([[100.]]) # Stupid way of simulating padding. TODO: do it in the right way
    target = torch.cat((target,tmp), dim=1)
    print(f"{src.shape=}")
    print(f"{target.shape=}")
    out = model(src,target)
    print(out.shape)
    break
    #print()

src.shape=torch.Size([1, 6])
target.shape=torch.Size([1, 6])
torch.Size([1, 28996])


In [39]:

if __name__ == "__main__":
    n_classes = 100

    source = torch.randint(low=0, high=n_classes, size=(20, 16))
    target = torch.randint(low=0, high=n_classes, size=(20, 16))

    print(source.shape)
    print(target.shape)

    s2s = TransformerModel(n_classes,700)

    out = s2s(source, target)
    print(out.size())
    print(out)

torch.Size([20, 16])
torch.Size([20, 16])
torch.Size([20, 100])
tensor([[0.0451, 0.0556, 0.0558,  ..., 0.0482, 0.0348, 0.0387],
        [0.0427, 0.0413, 0.0590,  ..., 0.0397, 0.0570, 0.0305],
        [0.0482, 0.0344, 0.0480,  ..., 0.0398, 0.0378, 0.0503],
        ...,
        [0.0509, 0.0518, 0.0477,  ..., 0.0434, 0.0547, 0.0864],
        [0.0552, 0.0601, 0.0335,  ..., 0.0601, 0.0336, 0.0401],
        [0.0303, 0.0615, 0.0523,  ..., 0.0573, 0.0531, 0.0465]],
       grad_fn=<SliceBackward0>)
